In [49]:
import pymysql
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
pd.set_option('max_rows',1000)

In [2]:
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

replace_puncs = lambda x: re.sub(r'[^\w\s]','',x)
remove_blanks = lambda x: re.sub(r'\s{2,}','',x)
cut_text = lambda x: list(jieba.cut(x,cut_all=False))

In [3]:
conn = pymysql.connect(host = '127.0.0.1', 
                       user = 'root', 
                       password = 'root123', 
                       db = 'delidou')

df = pd.read_sql_query('select link_id, name, tag from base_goods', conn)
df['clean_name'] = df.name.apply(remove_blanks).apply(replace_puncs).apply(find_chinese)
df['cutted_text'] = df.clean_name.apply(cut_text)
df['texts'] = df.cutted_text.apply(lambda x:' '.join(x))
tagged_df = df[df.tag != '']
untagged_df = df[df.tag == '']

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jh/q3qgwx4n59b_rg88dkbcb20m0000gn/T/jieba.cache
Loading model cost 0.609 seconds.
Prefix dict has been built succesfully.


In [4]:
id2label = {i:t for i,t in enumerate(tagged_df.tag.unique())}
label2id = { v:k for k,v in id2label.items()}
tagged_df ['label'] = tagged_df.tag.replace(label2id)

/Users/huan/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
vectorizer = TfidfVectorizer(max_features= 4000,ngram_range=(1,3))
vectorizer.fit(tagged_df.texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=4000,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [27]:
transformed_text = vectorizer.transform(tagged_df.texts).toarray()

In [28]:
transformed_text.shape

(125658, 4000)

In [29]:
X_train,X_test, y_train,y_test = train_test_split(transformed_text,tagged_df.label.values, test_size = 0.15, random_state = 42)

In [30]:
bayes = MultinomialNB()
bayes.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
bayes.score(X_test, y_test)

0.8826462942331158

In [44]:
predictions = bayes.predict(vectorizer.transform(untagged_df.texts).toarray())

In [45]:
labels = [id2label.get(p) for p in predictions]

In [46]:
untagged_df['pred_tag'] = labels

/Users/huan/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
pred_train = bayes.predict(vectorizer.transform(tagged_df.texts).toarray())

In [53]:
labels_train = [id2label.get(p) for p in pred_train]

In [54]:
tagged_df['pred_tag'] = labels_train

/Users/huan/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
tagged_df[tagged_df.tag!=tagged_df.pred_tag]

,link_id,name,tag,clean_name,cutted_text,texts,label,pred_tag
2,3ecdd3da072894bf5e34d56953bf128e,荞麦面低糖健康食品粗粮挂面杂粮冷面凉面 无糖食品 多规格包邮,其他,荞麦面低糖健康食品粗粮挂面杂粮冷面凉面无糖食品多规格包邮,"[荞麦面, 低糖, 健康, 食品, 粗粮, 挂面, 杂粮, 冷面, 凉面, 无糖, 食品, ...",荞麦面 低糖 健康 食品 粗粮 挂面 杂粮 冷面 凉面 无糖 食品 多 规格 包邮,1,食品
13,2f1508c15dfd41a0809c3330dfd7aa49,梳子女士专用长发气囊气垫梳卷发梳木梳网红款男头皮按摩头部经络,百货,梳子女士专用长发气囊气垫梳卷发梳木梳网红款男头皮按摩头部经络,"[梳子, 女士, 专用, 长发, 气囊, 气垫, 梳, 卷发, 梳, 木梳, 网红款, 男,...",梳子 女士 专用 长发 气囊 气垫 梳 卷发 梳 木梳 网红款 男 头皮 按摩 头部 经络,0,彩妆
41,aef26d5904e29baae0726e3654935877,信佳儿童手表女孩男孩防水夜光电子表 小孩学生数字式可爱男女童,饰品,信佳儿童手表女孩男孩防水夜光电子表小孩学生数字式可爱男女童,"[信佳, 儿童, 手表, 女孩, 男孩, 防水, 夜光, 电子表, 小孩, 学生, 数字式,...",信佳 儿童 手表 女孩 男孩 防水 夜光 电子表 小孩 学生 数字式 可爱 男 女童,5,母婴
70,72ff3531baa13fb829561465d79f85ab,电子手表男学生初中潮流防水高中特种兵机械男孩青少年运动电子表,饰品,电子手表男学生初中潮流防水高中特种兵机械男孩青少年运动电子表,"[电子手表, 男, 学生, 初中, 潮流, 防水, 高中, 特种兵, 机械, 男孩, 青少年...",电子手表 男 学生 初中 潮流 防水 高中 特种兵 机械 男孩 青少年 运动 电子表,5,男装
111,323881b9c6ef34a0021960ad39718e1c,2019新款塔罗牌印花纯棉短袖女宽松bf网红T恤欧美潮牌情侣装夏装,女装,新款塔罗牌印花纯棉短袖女宽松网红恤欧美潮牌情侣装夏装,"[新款, 塔罗牌, 印花, 纯棉, 短袖, 女, 宽松, 网红恤, 欧美, 潮牌, 情侣装,...",新款 塔罗牌 印花 纯棉 短袖 女 宽松 网红恤 欧美 潮牌 情侣装 夏装,9,男装
...,...,...,...,...,...,...,...,...
181288,d7cedd9ae58a83645a556cb80907e8bb,全棉时代氨基酸泡泡一次性洗脸巾纯棉擦脸巾深层清洁卸妆洁面湿巾,百货,全棉时代氨基酸泡泡一次性洗脸巾纯棉擦脸巾深层清洁卸妆洁面湿巾,"[全棉, 时代, 氨基酸, 泡泡, 一次性, 洗脸, 巾, 纯棉, 擦, 脸巾, 深层, 清...",全棉 时代 氨基酸 泡泡 一次性 洗脸 巾 纯棉 擦 脸巾 深层 清洁 卸妆 洁面 湿巾,0,彩妆
181299,ef197481b0775edad047a724da51ece8,肆方食斋芝麻夹心海苔脆儿童即食海苔宝宝孕妇零食办公室宿舍40g,食品,肆方食斋芝麻夹心海苔脆儿童即食海苔宝宝孕妇零食办公室宿舍,"[肆方, 食斋, 芝麻, 夹心, 海苔, 脆, 儿童, 即食, 海苔, 宝宝, 孕妇, 零食...",肆方 食斋 芝麻 夹心 海苔 脆 儿童 即食 海苔 宝宝 孕妇 零食 办公室 宿舍,12,生鲜
181304,f48d30ba710815d4ff6d521a589785f9,Swisse斯维诗血橙葡萄籽精华口服胶原蛋白液500ml维生素热巴同款,食品,斯维诗血橙葡萄籽精华口服胶原蛋白液维生素热巴同款,"[斯维诗, 血, 橙, 葡萄籽, 精华, 口服, 胶原蛋白, 液, 维生素, 热巴, 同款]",斯维诗 血 橙 葡萄籽 精华 口服 胶原蛋白 液 维生素 热巴 同款,12,护肤
181305,f4cc46758c9125c529ec13bf6c709ae9,一禅小和尚限量可爱卡通毛绒公仔玩偶周边礼物,百货,一禅小和尚限量可爱卡通毛绒公仔玩偶周边礼物,"[一禅, 小, 和尚, 限量, 可爱, 卡通, 毛绒, 公仔, 玩偶, 周边, 礼物]",一禅 小 和尚 限量 可爱 卡通 毛绒 公仔 玩偶 周边 礼物,0,母婴
